In [180]:
import json
import os
import pandas as pd
import string 
import pandas as pd 

from itertools import groupby
from operator import itemgetter

In [238]:
# VARIABLES
SPLIT = "train"

PATH_FC = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/vitaminc_factchecking/{}.jsonl".format(SPLIT)
PATH_REVISION = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/vitaminc_flagging/{}.jsonl".format(SPLIT)

SAMPLE = {
    "claim": "",
    "reference": "",
    "target": "",
    "score": None,
    "label": None
}

In [239]:
def load_jsonl_file(file_path, dataclass=None):
    content = []
    with open(file_path, "r", encoding="utf-8") as f:
        for entry in f.readlines():
            if dataclass:
                content.append(dacite.from_dict(data_class=dataclass, data=json.loads(entry)))
            else:
                content.append(json.loads(entry))
    return content


In [240]:
fc_data = load_jsonl_file(PATH_FC)
fc_revision = load_jsonl_file(PATH_REVISION)

In [241]:
df_fc_data = pd.DataFrame.from_dict(fc_data)

df_fc_data.head(3)

,unique_id,case_id,wiki_revision_id,label,claim,evidence,page,revision_type,FEVER_id
0,5ec5598dc9e77c000842c3cd_1,5ec5598dc9e77c000842c3cd,925488060,SUPPORTS,"Manchester had a population of more than 540,0...",Manchester ( ) is a major city and metropolita...,Manchester,real,NaN
1,5ec5598dc9e77c000842c3cd_2,5ec5598dc9e77c000842c3cd,925488060,NOT ENOUGH INFO,"Manchester had a population of more than 540,0...",Manchester ( ) is a major city and metropolita...,Manchester,real,NaN
2,5ec5598dc9e77c000842c3cd_3,5ec5598dc9e77c000842c3cd,925488060,NOT ENOUGH INFO,"Manchester had a population of less than 540,0...",Manchester ( ) is a major city and metropolita...,Manchester,real,NaN


### Create examples with score==0

In [242]:
# sort by multiple evidence 
fc_data_sorted = sorted(fc_data, key = itemgetter('claim'))
fc_data_grouped = groupby(fc_data_sorted, key = itemgetter('claim'))

In [243]:
for key, value in groupby(fc_data_sorted, key = itemgetter('claim')):
    print(key)
    for v in value:
        print(value)
    break

! n 1999 , Portia de Rossi was the winner of a Golden Globe Award .


In [244]:
def map_dataset_sample(ref_entry, target_entry, score):
    entry_output = SAMPLE.copy()
    entry_output['claim'] = target_entry['claim']
    entry_output['reference'] = "{} {}".format(target_entry['claim'], ref_entry['evidence'])
    entry_output['target'] = "{} {}".format(target_entry['claim'], target_entry['evidence'])
    entry_output['score'] = score
    entry_output['label'] = target_entry['label']
    
    return entry_output


In [245]:
def key_func(k):
    return k['claim']

i = 0
ds_samples_score0 = []
fc_data_sorted = sorted(fc_data, key = key_func)

for key, values in groupby(fc_data_sorted, key_func):        
    first_value = None
    for value in values:
        if not first_value: 
            first_value = value
            continue
        if value['label'] == first_value['label']:
            continue
        # only entries with different evidence such that label flips 
        ds_samples_score0.append(map_dataset_sample(first_value, value, score=0))

len(ds_samples_score0)

184802

In [226]:
# ds_samples_score0[:100]

### Create examples with score == 1

In [246]:
special_chars = [
    ",",
    ".",
    "(",
    ")",
    "'",
]

In [247]:
def add_whitespace(sent: str):
    for char in special_chars: 
        sent = sent.replace(char, " {} ".format(char)).replace("  ", " ")
    print(sent+"\n")
    return sent

def remove_spec_chars(sent: str):
    sent = re.sub(r"[^ a-zA-Z0-9]+",'', sent)
    return re.sub(' +', ' ', sent).strip()


In [248]:
def find_in_fc_data(evidence_sent, df): 
    evidence_sent = remove_spec_chars(evidence_sent)
    if evidence_sent in list(df['evidence_no_spec_chars']):
        return df.loc[df['evidence_no_spec_chars']==evidence_sent]
        
    return []

In [249]:
def map_dataset_sample_revision(fc_entry, revision_sent_a, revision_sent_b, score):
    fc_entry = pd.DataFrame(fc_entry)
#     print(fc_entry)
    
    mapped_entries = []
    for i, row in fc_entry.iterrows():
        entry_output = SAMPLE.copy()
        entry_output['claim'] = row['claim']
        entry_output['reference'] = "{} {}".format(row['claim'], revision_sent_a)
        entry_output['target'] = "{} {}".format(row['claim'], revision_sent_b)
        entry_output['score'] = score
        entry_output['label'] = row['label']
        # add to output list
        mapped_entries.append(entry_output)
    
    return mapped_entries


In [250]:
# remove spec chars to allow mapping
df_fc_data['evidence_no_spec_chars'] = df_fc_data['evidence'].apply(remove_spec_chars)

In [ ]:
# exmaples with non-factual changes
fc_revision_non_fact = [entry for entry in fc_revision if entry['label']=='not factual']
print("{} non-factual revisions out of {}".format(len(fc_revision_non_fact), len(fc_revision)))

ds_samples_score1 = []
for entry in fc_revision_non_fact:
    try:
        # find sentence a or b in fc_data
        match = find_in_fc_data(entry['sent_a'], df_fc_data)
        if len(match) == 0:
            match = find_in_fc_data(entry['sent_b'], df_fc_data)

        if len(match) > 0:
            # create example
            ds_samples_score1.append(map_dataset_sample_revision(match, entry['sent_a'], entry['sent_b'], score=1))
    except Exception as e: 
        print("Exception: {}".format(e))
        continue

154914 non-factual revisions out of 236998


In [ ]:
len(ds_samples_score1)

In [ ]:
ds_samples = ds_samples_score0 + ds_samples_score1
len(ds_samples)

### Save

In [ ]:
def to_dict(obj):
    return json.loads(json.dumps(obj, default=lambda o: o.__dict__))

def save_jsonl_file(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            json.dump(to_dict(entry), f)
            f.write("\n")


In [ ]:
output_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/reference_scorer_training_data/vitaminc_{}.jsonl".format(SPLIT)


In [ ]:
save_jsonl_file(ds_samples, output_path)